## Exercício 2
### Busque algum outro dataset no Kaggle para um problema de regressão e faça um novo treino. Lembre de modificar as métricas, ex.: MSE.

## Exercício 3 (Ative nas contants)
### Execute o MLFlow de maneira que se parar o container os dados não sejam perdidos, podendo salvar os dados ou no SQLite (default) ou outro banco de dados da sua escolha.

#### Import libs

In [ ]:
%pip install pandas
%pip install sklearn
%pip install mlflow
%pip install kagglehub

# Manipulação e visualização de dados
import pandas as pd
import numpy as np
import time

# Bibliotecas para aprendizado de máquina
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


# MLflow para gerenciamento de experimentos
import mlflow
from mlflow.models.signature import infer_signature
import subprocess

# Supressão de avisos
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings(
    "ignore",
    message=".*deprecated parameter 'name'.*",
    category=FutureWarning,
    module="mlflow",
)

# Datasets
import kagglehub
import shutil
import os
from pathlib import Path

#### Constants

In [ ]:
DATA_PATH = '../data/02-exercicio'
ENABLE_EXERCISE_3 = True
EXERCISE_3_MLFLOW_URI = "http://localhost:5001"
DOCKER_COMPOSE_FILE = "./03-mlflow/docker-compose.yml"

## Configura o MLFlow conforme necessário

In [ ]:


if ENABLE_EXERCISE_3:
    subprocess.run(["docker", "compose", "-f", "./03-mlflow/docker-compose.yml", "up", "-d"], check=True)
    mlflow.set_tracking_uri(EXERCISE_3_MLFLOW_URI)
else:
    mlflow.set_tracking_uri(None)

mlflow.set_experiment("bovespa_regression")


#### Utils

In [ ]:
def log_artifact(df: pd.DataFrame, name: str):
    path = f"{DATA_PATH}/{name}.csv"
    df.to_csv(path, index=False)
    mlflow.log_artifact(path, artifact_path="data")

def log_model(model, name: str):
    signature = infer_signature(X_test, y_pred)
    mlflow.sklearn.log_model(model, name=name, signature=signature, input_example=X_test.iloc[:3])

#### Download dataset

In [ ]:
dataset_path = kagglehub.dataset_download("andrewmvd/brazilian-stock-market")
dest_path = Path(DATA_PATH)

dest_path.mkdir(parents=True, exist_ok=True)

for file in Path(dataset_path).iterdir():
    shutil.copy(file, dest_path)

print(f"✅ Dataset copiado para: {dest_path.resolve()}")

df_bovespa_stocks_raw = pd.read_csv(f"{DATA_PATH}/bovespa_stocks.csv")
df_economic_indicators_raw = pd.read_csv(f"{DATA_PATH}/economic_indicators.csv")

#### EDA dos dados

In [ ]:
display(df_bovespa_stocks_raw.head())
df_bovespa_stocks_raw.info()

print("ITUB3 exists?", "ITUB3" in df_bovespa_stocks_raw["Symbol"].unique())

symbol_to_work = "ITUB3"

display(df_economic_indicators_raw.head())
df_economic_indicators_raw.info()


### Limpeza e Tratamento dos dados

#### Dados das ações

In [ ]:
with mlflow.start_run(run_name="stocks_treatment"):
    # Filtrando as ação que iremos trabalhar
    df_bovespa_stocks_clear = df_bovespa_stocks_raw[df_bovespa_stocks_raw["Symbol"] == symbol_to_work]
    mlflow.log_param("stocks symbol", symbol_to_work)

    # Transformando o time date
    df_bovespa_stocks_clear["Date"] = pd.to_datetime(df_bovespa_stocks_clear["Date"], format="ISO8601", utc=True)

    df_bovespa_stocks_clear["Date"] = pd.to_datetime(df_bovespa_stocks_clear["Date"]
                                                    .dt.tz_convert("America/Sao_Paulo")
                                                    .dt.tz_localize(None)
                                                    .dt.normalize())

    mlflow.log_param("Date conversion", "datetime:ISO8601:utc-True:convert-America/Sao_Paulo:localize-None:normalize")

    df_bovespa_stocks_clear = df_bovespa_stocks_clear.sort_values("Date").reset_index(drop=True)

    # Criando features derivadas
    df_bovespa_stocks_clear["year"] = df_bovespa_stocks_clear["Date"].dt.year
    df_bovespa_stocks_clear["month"] = df_bovespa_stocks_clear["Date"].dt.month
    df_bovespa_stocks_clear["day"] = df_bovespa_stocks_clear["Date"].dt.day
    df_bovespa_stocks_clear["weekday"] = df_bovespa_stocks_clear["Date"].dt.weekday

    df_bovespa_stocks_clear["return"] = df_bovespa_stocks_clear["Adj Close"].pct_change()
    df_bovespa_stocks_clear["MA7"] = df_bovespa_stocks_clear["Adj Close"].rolling(window=7).mean()
    df_bovespa_stocks_clear["MA30"] = df_bovespa_stocks_clear["Adj Close"].rolling(window=30).mean()

    mlflow.log_param("insert-Date.year", "dt.year")
    mlflow.log_param("insert-Date.month", "dt.month")
    mlflow.log_param("insert-Date.day", "dt.day")
    mlflow.log_param("insert-Date.weekday", "dt.weekday")
    mlflow.log_param("insert-Adj Close.return", "pct_changed()")
    mlflow.log_param("insert-Adj Close.MA7", "rolling(window=7).mean()")
    mlflow.log_param("insert-Adj Close.MA30", "rolling(window=30).mean()")


    # Removendo linhas onde as novas features com janelas temporais ficaram null
    df_bovespa_stocks_clear = df_bovespa_stocks_clear.dropna(subset=["return", "MA7", "MA30"]).reset_index(drop=True)
    mlflow.log_param("remove-null-return", True)
    mlflow.log_param("remove-null-MA7", True)
    mlflow.log_param("remove-null-MA30", True)

    # Padronização dos campos

    feature_to_scale_bovespa_stocks = ["Close", "High", "Low", "Open", "Volume", "return", "MA7", "MA30"]
    scaler = StandardScaler()
    scaled_values_bovespa_stocks = scaler.fit_transform(df_bovespa_stocks_clear[feature_to_scale_bovespa_stocks])
    scaled_cols_bovespa_stocks = [f"scaled_{col}" for col in feature_to_scale_bovespa_stocks]
    df_bovespa_stocks_clear[scaled_cols_bovespa_stocks] = scaled_values_bovespa_stocks
    mlflow.log_param("insert-Close", "StandardScaler")
    mlflow.log_param("insert-High", "StandardScaler")
    mlflow.log_param("insert-Low", "StandardScaler")
    mlflow.log_param("insert-Open", "StandardScaler")
    mlflow.log_param("insert-Volume", "StandardScaler")
    mlflow.log_param("insert-return", "StandardScaler")
    mlflow.log_param("insert-MA7", "StandardScaler")
    mlflow.log_param("insert-MA30", "StandardScaler")

    df_bovespa_stocks_clear = pd.get_dummies(df_bovespa_stocks_clear, columns=["year"], prefix="year_dummie")
    df_bovespa_stocks_clear = pd.get_dummies(df_bovespa_stocks_clear, columns=["month"], prefix="month_dummie")
    df_bovespa_stocks_clear = pd.get_dummies(df_bovespa_stocks_clear, columns=["day"], prefix="day_dummie")
    df_bovespa_stocks_clear = pd.get_dummies(df_bovespa_stocks_clear, columns=["weekday"], prefix="weekday_dummie")
    mlflow.log_param("insert-year", "dummies")
    mlflow.log_param("insert-month", "dummies")
    mlflow.log_param("insert-day", "dummies")
    mlflow.log_param("insert-weekday", "dummies")

    df_bovespa_stocks_clear.info()

#### Dados de indicadores

In [ ]:
with mlflow.start_run(run_name="indicators_treatment"):
    df_economic_indicators_clear = df_economic_indicators_raw.copy()
    df_economic_indicators_clear.info()

    # Transformando o time date
    df_economic_indicators_clear["Date"] = pd.to_datetime(df_economic_indicators_clear["Date"], format="ISO8601", utc=True)

    df_economic_indicators_clear["Date"] = pd.to_datetime(df_economic_indicators_clear["Date"]
                                                    .dt.tz_convert("America/Sao_Paulo")
                                                    .dt.tz_localize(None)
                                                    .dt.normalize())

    mlflow.log_param("Date conversion", "datetime:ISO8601:utc-True:convert-America/Sao_Paulo:localize-None:normalize")

    df_economic_indicators_clear = df_economic_indicators_clear.sort_values("Date").reset_index(drop=True)

    # Tratando valores nulos na serie temporal
    economic_cols_to_adjust = df_economic_indicators_clear.select_dtypes(include=["float", "int"]).columns
    df_economic_indicators_clear[economic_cols_to_adjust] = df_economic_indicators_clear[economic_cols_to_adjust].fillna(method="ffill").fillna(method="bfill")
    mlflow.log_param("imput-float_int", "fillna_ffill_bfill")

    # Padronização dos campos

    # Seleciona colunas numéricas (sem a data)
    economic_cols_to_scale = ["Taxa Selic", "IPCA", "IGP-M", "INPC", "Desemprego PNADC"]

    scaler = StandardScaler()
    economic_scaled_values = scaler.fit_transform(df_economic_indicators_clear[economic_cols_to_scale])

    economic_scaled_cols = [f"scaled_{col.replace(' ', '_')}" for col in economic_cols_to_scale]
    df_economic_indicators_clear[economic_scaled_cols] = economic_scaled_values

    mlflow.log_param("insert-Taxa Selic", "StandardScaler")
    mlflow.log_param("insert-IPCA", "StandardScaler")
    mlflow.log_param("insert-IGP-M", "StandardScaler")
    mlflow.log_param("insert-INPC", "StandardScaler")
    mlflow.log_param("insert-Desemprego PNADC", "StandardScaler")

#### Juntando as colunas e criando a minha de trabalho

In [ ]:
print(mlflow.__version__)

with mlflow.start_run(run_name="join_dataset"):
    df_clear = pd.merge_asof(
        df_bovespa_stocks_clear.sort_values("Date"),
        df_economic_indicators_clear.sort_values("Date"),
        on="Date",
        direction="backward"
    )

    mlflow.log_param("join_dataset-key", "Date")
    mlflow.log_param("join_dataset-unavailable_treatment", "backward")

    log_artifact(df_clear, name="processed_dataset")
    df_clear.info()



### Separação de dados de treino e teste

In [ ]:
with mlflow.start_run(run_name="split_dataset"):
    X = df_clear.drop(columns=["Date", "Symbol", "Adj Close", "Close", "High", "Low", "Open", "Volume", "return", "MA7", "MA30"])
    y = df_clear["Adj Close"]

    X.info()

    log_artifact(X, "X_dataset")
    log_artifact(y, "y_dataset")

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
    log_artifact(X_train, "X_train_dataset")
    log_artifact(X_test, "X_test_dataset")
    log_artifact(y_train, "y_train_dataset")
    log_artifact(y_test, "y_test_dataset")


### Definindo modelos a serem treinados

In [ ]:
models = [
    {
        "name": "Linear Regression",
        "model": LinearRegression,
        "params": [
            {},
            {
                "fit_intercept": False
            }
        ]
    },
    {
        "name": "Random Forest",
        "model": RandomForestRegressor,
        "params": [
            {
                "random_state": 42,
            },
            {
                "random_state": 42,
                "n_estimators": 500
            },
            {
                "random_state": 42,
                "max_depth": 10
            },
            {
                "random_state": 42,
                "n_estimators": 500,
                "max_depth": 10
            }
        ]
    },
    {
        "name": "Gradient Boosting",
        "model": GradientBoostingRegressor,
        "params": [
            {
                "random_state": 42,
            },
            {
                "random_state": 42,
                "n_estimators": 500
            },
            {
                "random_state": 42,
                "max_depth": 10
            },
            {
                "random_state": 42,
                "n_estimators": 500,
                "max_depth": 10
            }
        ]
    },
    {
        "name": "KNN Regressor",
        "model": KNeighborsRegressor,
        "params": [
            {},
            {
                "n_neighbors": 10
            }
        ]
    }
]

### Treina em diferentes modelos

In [ ]:
results = []

for item in models:
    model_name = item["name"]
    model_class = item["model"]

    for i, params in enumerate(item["params"], start=1):
        current_model_name = f"{model_name} | v{i}"
        print(f"Start train: {current_model_name}")
        model = model_class(**params)

        start_time = time.time()
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        end_time = time.time()

        # métricas
        mse        = mean_squared_error(y_test, y_pred)
        rmse       = np.sqrt(mse)
        mae        = mean_absolute_error(y_test, y_pred)
        r2         = r2_score(y_test, y_pred)
        train_time = end_time - start_time

        with mlflow.start_run(run_name=current_model_name):
            mlflow.log_param("model", model_name)
            mlflow.log_params(params)
            mlflow.log_metric("MSE", mse)
            mlflow.log_metric("RMSE", rmse)
            mlflow.log_metric("MAE", mae)
            mlflow.log_metric("R2", r2)
            mlflow.log_metric("train_time_s", train_time)
            log_model(model, model_name)

        results.append({
            "name": current_model_name,
            "MSE": mse, "RMSE": rmse, "MAE": mae, "R2": r2,
            "train time (s)": train_time,
            "model": model,
            "params": params
        })

        print(f"Stop train: {current_model_name}")


### Avalia resultados

In [ ]:
df_results = pd.DataFrame(results)

# Ordena: RMSE ↑prioridade, depois MAE, depois -R2 (maior é melhor), e por fim tempo
df_results = df_results.sort_values(
    by=["RMSE", "MAE", "R2", "train time (s)"],
    ascending=[True, True, False, True]
).reset_index(drop=True)

best = df_results.iloc[0]
print(df_results)
print(f"🏆 Melhor: {best['name']} | RMSE={best['RMSE']:.4f} | MAE={best['MAE']:.4f} | R²={best['R2']:.4f}")


### Salvando melhor modelo

In [ ]:
with mlflow.start_run(run_name="Best model"):
    mlflow.log_param("model", best["name"])
    mlflow.log_params(best["params"])

    mlflow.log_metric("MSE", best["MSE"])
    mlflow.log_metric("RMSE", best["RMSE"])
    mlflow.log_metric("MAE", best["MAE"])
    mlflow.log_metric("R2", best["R2"])
    mlflow.log_metric("train time", best["train time (s)"])
    log_model(model, name="Best model")
best_model_name = best["name"]
print(f"Melhor modelo ({best_model_name}) armazenado com sucesso no MLflow.")